In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import *
import csv


Read merged data file with charging stations and ACS data

In [4]:
#import the merged charging station and ACS data
# lines = open('data/add_0_stations.csv').read().splitlines()
# preprocessed the data file and removed missing values
# to do: use imputer to handle missing vaules in scikit
# http://stackoverflow.com/questions/30317119/classifiers-in-scikit-learn-that-handle-nan-null?lq=1
lines = open('data/missing_values_removed.csv').read().splitlines()
#Remove the first row, it's just the headings
lines.pop(0)

zip_code, count, population, male_pct, age, college_pct, income, home_pct = [], [], [], [], [], [], [], []
for line in lines:
    items = line.split(',')
    zip_code.append(items[0])
    count.append(float(items[1]))
    population.append(float(items[2]))
    male_pct.append(float(items[3]))
    age.append(float(items[4]))
    college_pct.append(float(items[5]))
    income.append(float(items[6]))
    home_pct.append(float(items[7]))
    
# combine lists to numpy arrays for data and labels
Y=np.array(count)
X=np.vstack((population,male_pct, age, college_pct, income, home_pct))
# X=np.vstack((population, male_pct, home_pct))
X=np.transpose(X)
print Y.shape, X.shape, X[0], Y[0]

# add shuffle code before splitting the data into train and dev sets
# easiest shuffle utility is sklearn shuffle function
from sklearn.utils import shuffle

# shuffle two arrays in unison
X, Y = shuffle(X, Y, random_state=0)
print X.shape, Y.shape, X[0], Y[0]
# determine the data type of the two arrays
print X.dtype, Y.dtype
# convert to strings
# must handle missing values first by making missing values zeros
X[X==''] = '0'
# now convert to float or leave them alone - converting now to make the code cleaner
X1=X.astype(np.float)
print X1.dtype

# Split the imported data into training 75% and development data 25%

split = 3*len(zip_code)/4

print len(zip_code), split

train_data, train_labels = X[:split], Y[:split]
dev_data, dev_labels = X[split:], Y[split:]

print train_data.shape, train_labels.shape
print dev_data.shape, dev_labels.shape
print train_data[0], train_labels[0]
print dev_data[0], dev_labels[0]
# dev_dat=np.array(dev_data)
# print dev_dat.sum()

(30514,) (30514, 6) [  6.40200000e+03   4.97188379e-01   1.94000000e+01   6.00000000e-01
   0.00000000e+00   0.00000000e+00] 1.0
(30514, 6) (30514,) [  1.21300000e+03   4.50123660e-01   3.97000000e+01   0.00000000e+00
   3.52120000e+04   8.70689655e-01] 0.0
float64 float64
float64
30514 22885
(22885, 6) (22885,)
(7629, 6) (7629,)
[ 0.  0.  0.  0.  0.  0.] 0.0
[  3.23560000e+04   5.01730745e-01   3.64000000e+01   2.18000000e+01
   4.57570000e+04   6.60303392e-01] 2.0


In [5]:
# try lineary regression mode
from sklearn import datasets, linear_model

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_data, train_labels)

# The coefficients
print('Coefficients: \n', regr.coef_)

# score
print('Variance score: %.2f' % regr.score(dev_data, dev_labels))


('Coefficients: \n', array([  2.06276543e-05,   7.74429448e-01,   7.38819633e-03,
         9.43223599e-03,   1.42421493e-05,  -1.71728883e+00]))
Variance score: 0.19


In [6]:
# fit the first model with logistic regression 
#
logreg = LogisticRegression(penalty='l1', C=1.0, tol=0.01) 
logreg.fit(train_data, train_labels)
print logreg.score(dev_data, dev_labels)

0.861580810067
